In [18]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd

In [19]:
spark = SparkSession.builder.appName("customer").getOrCreate()


In [20]:
df_cc= spark.read.load('json\cdw_sapp_credit.json', format='json',header=True,inferSchema=True)

In [21]:
df_cc.printSchema()

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)



lets change the data type of each date column ot string so as to convert into concat

In [22]:
#pyspark.sql.functions.lpad(col, len, pad)[source]


df_cc = df_cc.withColumn('MONTH', lpad(df_cc['MONTH'], 2, '0'))


In [23]:
df_cc = df_cc.withColumn('DAY', lpad(df_cc['DAY'], 2, '0'))


In [24]:
df_cc=df_cc.withColumn('DAY',df_cc['DAY'].cast('String'))
df_cc=df_cc.withColumn('MONTH',df_cc['MONTH'].cast('String'))
df_cc=df_cc.withColumn('YEAR',df_cc['YEAR'].cast('String'))

In [25]:
#concat to yyyyddmm format
df_cc=df_cc.withColumn('TIMEID',concat(df_cc['YEAR'],df_cc['MONTH'],df_cc['DAY']))

In [26]:
df_cc=df_cc.drop('DAY','MONTH','YEAR')

In [27]:
df_cc.printSchema()

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- TIMEID: string (nullable = true)



In [28]:
df_cc=df_cc.withColumnRenamed('CREDIT_CARD_NO','CUST_CC')

loading in db

In [29]:
df_cc.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
  .option("user", "root") \
  .option("password", "password") \
  .save()


In [30]:
df_new1 = spark.read.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
  .option("user", "root") \
  .option("password", "password") \
  .load()

df_new1.show()


+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|BRANCH_CODE|         CUST_CC| CUST_SSN|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|        180|4210653342242023|123451310|         45069|           Bills|            77.79|20180315|
|        114|4210653349028689|123459988|             1|       Education|             78.9|20180214|
|        156|4210653312478046|123455692|         22562|         Grocery|            91.08|20180813|
|        114|4210653312478046|123455692|         22563|           Bills|             22.2|20180626|
|         35|4210653349028689|123459988|             2|   Entertainment|            14.24|20180320|
|         47|4210653312478046|123455692|         22564|         Grocery|            77.54|20180320|
|        160|4210653349028689|123459988|             3|         Grocery|             56.7|20180708|
